In [11]:
%run "C:/Users/Varun Bhat/minimax.ipynb"
%run "C:/Users/Varun Bhat/mcts.ipynb"

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import random

directions =  [(-1 , -1), (-1, 0), (-1, 1), 
               (0, -1), (0, 1), 
               (1, -1), (1, 0), (1, 1)
              ]

def initialize_board():
    board = np.full((8,8), 0)
        
    board[3][4] = board[4][3] = 1
    board[3][3] = board[4][4] = 2
    return board

def switch_piece(piece):
    return 1 if piece==2 else 2

def place_piece(board, row, col, piece):
    state = copy.deepcopy(board)
    if __valid__(state, row, col, piece):
        state[row][col] = piece
        board_init = state
        row_ini = row
        col_ini = col
        
        for dir in directions:
            
            terminating_piece = False
            row += dir[0]
            col += dir[1]       
            to_flip = []
            
            while 0<=row<8 and 0<=col<8 and state[row][col] == switch_piece(piece):
                to_flip.append((row, col))
                row += dir[0]
                col += dir[1]                                    
                if 0<=row<8 and 0<=col<8 and state[row][col] == piece:
                    terminating_piece = True
                    
            if not terminating_piece:
                state = board_init
            if len(to_flip) and terminating_piece:
                for (ROW, COL) in to_flip:
                    state[ROW][COL] = piece
                    
            row = row_ini
            col = col_ini
                
    else:        
        return False
    return state

def get_piece(state, row, col):
    return state[row][col]

def __valid_positions__(state, piece):
    positions = set()
    for col in range(8):
        for row in range(8):
            if state[row][col] == 0:
                row_ini, col_ini = row, col
                for dir in directions:                        
                    row += dir[0]
                    col += dir[1]
                    while 0<=row<8 and 0<=col<8 and state[row][col] == switch_piece(piece):
                        row += dir[0]
                        col += dir[1]
                        if 0<=row<8 and 0<=col<8 and state[row][col] == piece:
                            positions.add((row_ini, col_ini)) 
                            break
                    row, col = row_ini, col_ini
    return list(positions)
        
def __valid__(state, row, col, piece):
    return (row, col) in __valid_positions__(state, piece)

def terminate(state, running):
    if not __valid_positions__(state, 1) and not __valid_positions__(state, 2):
        white_count = np.sum(state == 2)
        black_count = np.sum(state == 1)
        if white_count > black_count:
            print("White wins")
        elif black_count < white_count:
            print("Black wins")  
        else:
            print("It is a tie")
        running = False
    return running

In [13]:
import pygame
import sys

AI = AlphaBetaMinimax

pygame.init()

WIDTH, HEIGHT = 400, 400  
CELL_SIZE = WIDTH // 8     
BACKGROUND_COLOR = (0, 100, 0)  
LINE_COLOR = (255, 255, 255)     
BLACK_COLOR = (0, 0, 0)          
WHITE_COLOR = (255, 255, 255)    
HIGHLIGHT_COLOR = (255, 215, 0)  

color_mapping = {0 : None,
                 1 : (0, 0, 0), 
                 2 : (255, 255, 255)
                }
player_mapping = {"W" : 2,
                  "B" : 1
                 }

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Othello Game")

board = initialize_board()
player = input("[W/B] ?")
current_turn = 1 

def draw_board():
    screen.fill(BACKGROUND_COLOR)
    
    for x in range(1, 8):
        pygame.draw.line(screen, LINE_COLOR, (x * CELL_SIZE, 0), (x * CELL_SIZE, HEIGHT))
        pygame.draw.line(screen, LINE_COLOR, (0, x * CELL_SIZE), (WIDTH, x * CELL_SIZE))
    
    for row in range(8):
        for col in range(8):
            if board[row][col]:
                pygame.draw.circle(screen, color_mapping[board[row][col]], (col * CELL_SIZE + CELL_SIZE // 2, row * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 2)

    mouse_pos = pygame.mouse.get_pos()
    row, col = mouse_pos[1] // CELL_SIZE, mouse_pos[0] // CELL_SIZE
    if color_mapping[board[row][col]] is None:
        pygame.draw.rect(screen, HIGHLIGHT_COLOR, (col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE), 2)

    pygame.display.flip()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False    
        if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:  
                    running = False 
                    pygame.quit()
                    sys.exit()

        if player == 'B': 
            if current_turn == 1:
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    row, col = pos[1] // CELL_SIZE, pos[0] // CELL_SIZE
                    if __valid__(board, row, col, current_turn):
                        board = place_piece(board, row, col, current_turn)
                        current_turn = switch_piece(current_turn) if len(__valid_positions__(board, switch_piece(current_turn))) != 0 else current_turn
                    else:
                        print("Invalid Move")
            else:
                Agent = AI(board, 2)
                move = Agent.return_move()
                board = place_piece(board, move[0], move[1], current_turn)
                current_turn = switch_piece(current_turn) if len(__valid_positions__(board, switch_piece(current_turn))) != 0 else current_turn
        else:
            if current_turn == 1:
                Agent = AI(board, 1)
                move = Agent.return_move()
                board = place_piece(board, move[0], move[1], current_turn)
                current_turn = switch_piece(current_turn) if len(__valid_positions__(board, switch_piece(current_turn))) != 0 else current_turn
            else:
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()
                    row, col = pos[1] // CELL_SIZE, pos[0] // CELL_SIZE
                
                    if __valid__(board, row, col, current_turn):
                        board = place_piece(board, row, col, current_turn)
                        current_turn = switch_piece(current_turn) if len(__valid_positions__(board, switch_piece(current_turn))) != 0 else current_turn
                    else:
                        print("Invalid Move")
            
    running = terminate(board, running) 
    draw_board()

pygame.quit()
sys.exit()

[W/B] ? W


Move played :  (2, 3)
Invalid Move
Move played :  (3, 2)


SystemExit: 